In [22]:
import sys
import itertools
from anytree import Node, RenderTree,Resolver,ChildResolverError,NodeMixin,PreOrderIter
from anytree.dotexport import RenderTreeGraph

class CustomNode(NodeMixin):  # Add state number feature
    def __init__(self, name, state_number,symbol,end_state, parent=None):
        super(CustomNode, self).__init__()
        self.name=name
        self.state_number=state_number
        self.parent=parent
        self.end_state=end_state
        self.symbol=symbol

state_number=0
top=CustomNode("root",state_number,'',False)
r = Resolver('name')




In [23]:
def add_inflectional_suffixes(suffix,state_number,root_node):
    suffixes=suffix.split(":")
    state_number+=1
    
    node=CustomNode(suffixes[0],state_number,suffixes[1],True,root_node)
    if suffixes[1]=='+N':
        state_number+=1
        plural_node=CustomNode('^s',state_number,'+PL',True,node)
        state_number+=1
        CustomNode("^s",state_number,'+SGPS',True,node)
        state_number+=1
        CustomNode("'",state_number,'+PLPS',True,plural_node)
    elif suffixes[1]=='+V':
        state_number+=1
        CustomNode("^s",state_number,'+V+3D',True,node)
        state_number+=1
        CustomNode("ing",state_number,'+ING',True,node)
    return (node,state_number)
def append_suffixes(root_node,morphs,index,state_number):
    suffix=''
    while len(morphs)>index:
        index+=1
        if morphs[index]=='|':
            if suffix!='': #check if suffix to be handle before pipe
                
                node,state_number=add_inflectional_suffixes(suffix,state_number,root_node)
                suffix=''
        elif morphs[index]=='(':
            node,state_number=add_inflectional_suffixes(suffix,state_number,root_node)
            index,state_number=append_suffixes(node,morphs,index,state_number)
            suffix=''
        elif morphs[index]==')':
            if suffix!='': #check if suffix to be handle before paranthesis
                val=add_inflectional_suffixes(suffix,state_number,root_node)
                state_number=val[1]
            return (index,state_number)
        else:
            suffix+=morphs[index]
    return (index,state_number)

def insert_irregular_noun(stem,state_number,root):
    verbs=stems=stem.split('/')
    root_stem=verbs[0]
    for i in range(len(root_stem)):
        state_number+=1
        root=CustomNode(stem[i],state_number,'<eps>',False,root)
        
    state_number+=1
    n=CustomNode(verbs[0],state_number,verbs[0]+'+N+SG',True,root)
    
    state_number+=1
    n=CustomNode(verbs[1],state_number,verbs[1]+'+N+PL',True,root)
            
    return (root,state_number)

In [24]:
#words = ['compute (er:+N(ize:+V(ation:+N|able:+ADJ)|less:+ADJ(ness:+N))|ation:+N(able:+ADJ))','work (er:+N(ism:+N))']
#words = ["be/were&was/been (ing:+N)","begin/began/begun (ing:+N|er:+N)"]
with open('Lexicon/Nouns.txt') as f:
    words = f.readlines()
    words.sort()
    for word in words:
        line=word.rstrip()
        arr=line.split(' ')
        stem=arr[0]
        if len(arr)==2:
            morphs=arr[1]
        root=top

        if '/' in stem: #means irregular verb
            root,state_number=insert_irregular_noun(stem,state_number,top)
        else:
            for i in range(len(stem)):
                try:
                    n=r.get(root,stem[i])
                except ChildResolverError:
                    state_number+=1
                    n=CustomNode(stem[i],state_number,stem[i],False,root)
                root=n
            state_number+=1
            CustomNode("<eps>",state_number,'+N+SG',True,root)
            state_number+=1
            CustomNode("^s",state_number,'+N+PL',True,root)
        
        if len(arr)==2:
            return_val=append_suffixes(root,morphs,0,state_number)
            state_number=return_val[1]
RenderTreeGraph(top).to_picture("udo.png")

In [25]:
p=PreOrderIter(top)

p=itertools.islice(p,1,None) #pass root


for n in p:
    print(n.name + " " +str(n.state_number) + "parent: " + n.parent.name)

a 1parent: root
c 2parent: a
t 3parent: c
<eps> 4parent: t
^s 5parent: t
i 6parent: a
m 7parent: i
<eps> 8parent: m
^s 9parent: m
n 10parent: a
s 11parent: n
w 12parent: s
e 13parent: w
r 14parent: e
<eps> 15parent: r
^s 16parent: r
r 17parent: a
t 18parent: r
<eps> 19parent: t
^s 20parent: t
-ist 21parent: t
^s 22parent: -ist
' 24parent: ^s
^s 23parent: -ist
-ry 25parent: -ist
^s 26parent: -ry
' 28parent: ^s
^s 27parent: -ry
-ic 29parent: -ist
-al 30parent: -ic
-y 31parent: t
t 32parent: a
t 33parent: t
a 34parent: t
c 35parent: a
k 36parent: c
<eps> 37parent: k
^s 38parent: k
b 39parent: root
a 40parent: b
n 41parent: a
<eps> 42parent: n
^s 43parent: n
d 44parent: root
a 45parent: d
t 46parent: a
a 47parent: t
data 48parent: a
data 49parent: a
e 50parent: root
s 51parent: e
c 52parent: s
a 53parent: c
p 54parent: a
e 55parent: p
<eps> 56parent: e
^s 57parent: e
h 58parent: root
a 59parent: h
t 60parent: a
e 61parent: t
<eps> 62parent: e
^s 63parent: e
i 64parent: h
s 65parent: i
t 66

In [26]:

#for pre, _, node in RenderTree(top):
#    treestr = u"%s%s" % (pre, node.name)
#    print(treestr.ljust(8), node.state_number)
def print_as_tabs(prev_state,next_state,input_label,output_label):
    print(str(prev_state) + "\t" + str(next_state) + "\t"+ str(input_label) + "\t" + str(output_label))


# In[222]:


p=PreOrderIter(top)
p=itertools.islice(p,1,None) #pass root

print_as_tabs(0,1,"<s>","<s>")
print_as_tabs(1,0,"<eps>","<eps>")
print_as_tabs(0,2,"</s>","</s>")
print_as_tabs(2,0,"<eps>","<eps>")
print_as_tabs(0,3,"<unk>","<unk>")
print_as_tabs(3,0,"<eps>","<eps>")
for n in p:
    parent_state_number=n.parent.state_number
    if n.parent.state_number!=0:
        parent_state_number+=3
    print_as_tabs(parent_state_number,n.state_number+3,n.symbol,n.name)
    if n.end_state:
        print_as_tabs(n.state_number+3,0,"<eps>","#")  
    #if n.is_leaf:
     #   print_as_tabs(n.state_number+3,0,"<eps>","#")
print("0")


0	1	<s>	<s>
1	0	<eps>	<eps>
0	2	</s>	</s>
2	0	<eps>	<eps>
0	3	<unk>	<unk>
3	0	<eps>	<eps>
0	4	a	a
4	5	c	c
5	6	t	t
6	7	+N+SG	<eps>
7	0	<eps>	#
6	8	+N+PL	^s
8	0	<eps>	#
4	9	i	i
9	10	m	m
10	11	+N+SG	<eps>
11	0	<eps>	#
10	12	+N+PL	^s
12	0	<eps>	#
4	13	n	n
13	14	s	s
14	15	w	w
15	16	e	e
16	17	r	r
17	18	+N+SG	<eps>
18	0	<eps>	#
17	19	+N+PL	^s
19	0	<eps>	#
4	20	r	r
20	21	t	t
21	22	+N+SG	<eps>
22	0	<eps>	#
21	23	+N+PL	^s
23	0	<eps>	#
21	24	+N	-ist
24	0	<eps>	#
24	25	+PL	^s
25	0	<eps>	#
25	27	+PLPS	'
27	0	<eps>	#
24	26	+SGPS	^s
26	0	<eps>	#
24	28	+N	-ry
28	0	<eps>	#
28	29	+PL	^s
29	0	<eps>	#
29	31	+PLPS	'
31	0	<eps>	#
28	30	+SGPS	^s
30	0	<eps>	#
24	32	+ADJ	-ic
32	0	<eps>	#
32	33	+ADJ	-al
33	0	<eps>	#
21	34	+ADJ	-y
34	0	<eps>	#
4	35	t	t
35	36	t	t
36	37	a	a
37	38	c	c
38	39	k	k
39	40	+N+SG	<eps>
40	0	<eps>	#
39	41	+N+PL	^s
41	0	<eps>	#
0	42	b	b
42	43	a	a
43	44	n	n
44	45	+N+SG	<eps>
45	0	<eps>	#
44	46	+N+PL	^s
46	0	<eps>	#
0	47	<eps>	d
47	48	<eps>	a
48	49	<eps>	t
49	50	<eps>	a
50	51	data+N+SG	data
5

In [30]:
def append_suffixes(node_stack,morphs,index,paranthesis_stack,state_number):
    suffix=''
    while len(paranthesis_stack)!=0:
        index+=1
        
        if morphs[index]=='|':
            if suffix!='':
                node=node_stack[-1] 
                state_number+=1
                suffixes=suffix.split(':')
                CustomNode(suffixes[0],state_number,suffixes[1],node)
                suffix=''
        elif morphs[index]=='(':
            suffixes=suffix.split(':')
            state_number+=1
            
            node=CustomNode(suffixes[0],state_number,suffixes[1],node_stack[-1])
            node_stack.append(node)
            paranthesis_stack.append(index)
            param_node_stack=[node_stack[-1]]
            param_paranthesis_stack=[index]
            print(node_stack[-1].name)
            print(param_paranthesis_stack)
        
            var=append_suffixes(param_node_stack,morphs,index,param_paranthesis_stack,state_number)
            index=var[0]
            state_number=var[1]
            node_stack.pop()
            paranthesis_stack.pop()
            suffix=''
            print(node_stack[-1].name)
        elif morphs[index]==')':
            if suffix!='':
                node=node_stack[-1] 
                state_number+=1
                suffixes=suffix.split(':')
                CustomNode(suffixes[0],state_number,suffixes[1],node)
            paranthesis_stack.pop()
            node_stack.pop()
            return (index,state_number,node_stack,paranthesis_stack)
        else:
            suffix+=morphs[index]
    return (index,state_number,node_stack,paranthesis_stack)